# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
```
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [2]:
import pandas as pd
import numpy as np
import requests

# 1) seu código aqui
sinasc = pd.read_csv('/content/SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(18336, 69)


(18336, 69)

In [3]:
# 2) Conte o número de valores missing por variável

sinasc.isnull().sum()

ORIGEM         0
CODESTAB      77
CODMUNNASC     0
LOCNASC        0
IDADEMAE       0
              ..
munResUf       1
munResLat      2
munResLon      2
munResAlt      2
munResArea     2
Length: 69, dtype: int64

In [4]:
# 3) Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
#['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

sinascImportante = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

sinascImportante.head(5)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [5]:
# 4) Apgar é uma nota que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento.
# Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de missings.

# A coluna sinascImportante['APGAR5'].isnull().sum() tem 46 resultados nulos ou não preenchidos

sinascAPGAR5 = sinascImportante.dropna(subset=['APGAR5'])



In [20]:
# 5) observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9.
sinascAPGAR5['ESTCIVMAE'].fillna(9, inplace=True)
sinascAPGAR5['CONSULTAS'].fillna(9, inplace=True)


sinascAPGAR5.isnull().sum()

<ipython-input-20-775e74d96c62>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5['ESTCIVMAE'].fillna(9, inplace=True)
<ipython-input-20-775e74d96c62>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5['CONSULTAS'].fillna(9, inplace=True)


LOCNASC         0
IDADEMAE        0
ESTCIVMAE       0
ESCMAE        183
QTDFILVIVO      0
GESTACAO      811
GRAVIDEZ       44
CONSULTAS       0
APGAR5          0
ASFIXIA         0
dtype: int64

In [21]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.

sinascAPGAR5['QTDFILVIVO'].fillna(0, inplace=True)
sinascAPGAR5.isnull().sum()

<ipython-input-21-51ca65139a78>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5['QTDFILVIVO'].fillna(0, inplace=True)


LOCNASC         0
IDADEMAE        0
ESTCIVMAE       0
ESCMAE        183
QTDFILVIVO      0
GESTACAO      811
GRAVIDEZ       44
CONSULTAS       0
APGAR5          0
ASFIXIA         0
dtype: int64

In [22]:
# 7) Das restantes, decida que valores te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha.
# Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.

sinascAPGAR5.isnull().sum()

# aqui podemos perceber que as colunas  ESCMAE, GESTACAO, GRAVIDEZ


LOCNASC         0
IDADEMAE        0
ESTCIVMAE       0
ESCMAE        183
QTDFILVIVO      0
GESTACAO      811
GRAVIDEZ       44
CONSULTAS       0
APGAR5          0
ASFIXIA         0
dtype: int64

In [23]:
# Vamos analisar as porcetagens dos valores por coluna

print(sinascAPGAR5['ESCMAE'].value_counts() / len(sinascAPGAR5['ESCMAE']) * 100)
print('Valores da coluna ESCMAE\n')
print(sinascAPGAR5['GESTACAO'].value_counts() / len(sinascAPGAR5['GESTACAO']) * 100)
print('Valores da coluna GESTACAO\n')
print(sinascAPGAR5['GRAVIDEZ'].value_counts() / len(sinascAPGAR5['GRAVIDEZ']) * 100)
print('Valores da coluna GRAVIDEZ\n')



8 a 11 anos        58.230273
12 anos ou mais    21.079129
4 a 7 anos         17.697275
1 a 3 anos          1.838678
Nenhuma             0.153223
Name: ESCMAE, dtype: float64
Valores da coluna ESCMAE

37 a 41 semanas        83.747401
32 a 36 semanas         8.224800
42 semanas e mais       2.692350
28 a 31 semanas         0.563642
22 a 27 semanas         0.306446
Menos de 22 semanas     0.027361
Name: GESTACAO, dtype: float64
Valores da coluna GESTACAO

Única              98.051877
Dupla               1.701872
Tríplice e mais     0.005472
Name: GRAVIDEZ, dtype: float64
Valores da coluna GRAVIDEZ



In [24]:
# podemos perceber que para as colunas GESTACAO, GRAVIDEZ  podemos facilmente substituir os valors pelo valor que aparece em mair portagem porque dessa forma não alteramos significativamente o resultado

sinascAPGAR5['GESTACAO'].fillna('37 a 41 semanas', inplace=True)

sinascAPGAR5['GRAVIDEZ'].fillna('Única', inplace=True)

sinascAPGAR5.isnull().sum()


<ipython-input-24-15178e3a1307>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5['GESTACAO'].fillna('37 a 41 semanas', inplace=True)
<ipython-input-24-15178e3a1307>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5['GRAVIDEZ'].fillna('Única', inplace=True)


LOCNASC         0
IDADEMAE        0
ESTCIVMAE       0
ESCMAE        183
QTDFILVIVO      0
GESTACAO        0
GRAVIDEZ        0
CONSULTAS       0
APGAR5          0
ASFIXIA         0
dtype: int64

In [28]:
# para a coluna ESCMAE eu fazr os valores serem substituídos de forma randomizada entre os três valores de maior porcentagem

# vou definir uma função para trazer os valores randômicos que eu quero

import random

def random_fill(x):
    return random.choice(['8 a 11 anos', '12 anos ou mais', '4 a 7 anos'])

sinascAPGAR5['ESCMAE'] = sinascAPGAR5['ESCMAE'].fillna(random_fill)

sinascAPGAR5.isnull().sum()

<ipython-input-28-16a1f109fa32>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5['ESCMAE'] = sinascAPGAR5['ESCMAE'].fillna(random_fill)


LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
ASFIXIA       0
dtype: int64

In [11]:
# 8) O Apgar possui uma classificação indicando se o bebê passou por asfixia:
# Entre 8 e 10 está em uma faixa 'normal'.
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
# Entre 4 e 5 significa 'asfixia moderada'.
# Entre 0 e 3 significa 'asfixia severa'.
# Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.


sinascAPGAR5.loc[sinascAPGAR5['APGAR5'] <= 3, 'ASFIXIA'] = 'asfixia severa'
sinascAPGAR5.loc[(sinascAPGAR5['APGAR5'] > 3) & (sinascAPGAR5['APGAR5'] <= 5), 'ASFIXIA'] = 'asfixia moderada'
sinascAPGAR5.loc[(sinascAPGAR5['APGAR5'] > 5) & (sinascAPGAR5['APGAR5'] <= 7), 'ASFIXIA'] = 'asfixia leve'
sinascAPGAR5.loc[(sinascAPGAR5['APGAR5'] > 7) & (sinascAPGAR5['APGAR5'] <= 10), 'ASFIXIA'] = 'normal'

sinascAPGAR5.tail()


<ipython-input-11-ed08a70d2f1f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5.loc[sinascAPGAR5['APGAR5'] <= 3, 'ASFIXIA'] = 'asfixia severa'


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,ASFIXIA
18331,1,31,5.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,9.0,normal
18332,1,23,5.0,8 a 11 anos,1.0,42 semanas e mais,Única,3.0,9.0,normal
18333,1,26,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4.0,9.0,normal
18334,1,14,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,9.0,normal
18335,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0,normal


In [12]:
sinascAPGAR5['ASFIXIA'].value_counts()

normal              17958
asfixia leve          219
asfixia severa         49
asfixia moderada       48
Name: ASFIXIA, dtype: int64

In [13]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palavras.
# Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.

sinascAPGAR5['ASFIXIA'].replace(['normal', 'asfixia leve', 'asfixia moderada', 'asfixia severa'], ['normal', 'asfixia_leve', 'asfixia_moderada', 'asfixia_severa'], inplace=True)


<ipython-input-13-145a19733b3f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinascAPGAR5['ASFIXIA'].replace(['normal', 'asfixia leve', 'asfixia moderada', 'asfixia severa'], ['normal', 'asfixia_leve', 'asfixia_moderada', 'asfixia_severa'], inplace=True)


In [14]:
sinascAPGAR5.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,ASFIXIA
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0,normal
2,1,37,NaN,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0,normal
